# Mapeo de Hénon

- Hénon propuso, en 1976, un mapeo en 2D sencillo que exhibiera las propiedades esenciales del sistema de ecuaciones de primer orden acopladas propuestas por Lorentz, con el objetivo de poder estudiar de forma sencilla dichas propiedades.

- El mapeo imita una sección de Poincaré del atractor de Lorentz.

- El mapeo propuesto es el siguiente 

\begin{equation}
\begin{split}
    T:\,&x_{i+1}=y_{i+1}-ax_{i}^{2}\\
    &y_{i+1}=bx_{i}
\end{split}
\end{equation}





- Dicho mapeo se puede descomponer en tres etapas, i.e., $T\,=\,T'''T''T$. Si se toma como condición inicial una elipse:


![](doblamiento_T1.png)

- La primera etapa consiste en un doblamiento:

\begin{equation}
\begin{split}
    T':\,&x'=x\\
    &y'=y+1-ax^{2}
\end{split}
\end{equation}

![](doblamiento_T2.png)

- La segunda etapa consiste en una contracción en $x$:

\begin{equation}
\begin{split}
    T'':\,&x''=bx'\\
    &y''=y
\end{split}
\end{equation}


![](doblamiento_T3.png)

- La tercera etapa consiste en un reorientación en el eje x:

\begin{equation}
\begin{split}
    T''':\,&x'''=y''\\
    &y'''=x''
\end{split}
\end{equation}

![](doblamiento_T4.png)

- Es un mapeo con Jacobiano constante:

\begin{equation}
    \frac{\partial (x_{i+1},y_{i+1})}{\partial (x_{i},y_{i})}=-b
\end{equation}



- Tiene dos puntos invariantes

\begin{equation}
\begin{split}
    &x=\frac{-(1-b)\pm\sqrt{(1-b)^{2}+4a}}{2a}\\
    &y=bx
\end{split}
\end{equation}



- Dichos puntos son reales para 
\begin{equation}
a>a_{0}=-\frac{1}{4}(1-b)^{2}.
\end{equation}


En este caso, un punto es siempre linealmente inestable, mientras que el otro es inestable para 

\begin{equation}
a>a_{1}=\frac{3}{4}(1-b)^{2}.
\end{equation}

# Áreas

# Caso conservativo 

- En este caso $b=-1$ y $b=1$. Para que existan puntos fijos reales se debe cumplir $a>a_{0}=-\frac{1}{4}(1-b)^{2}$.

- A continuación se presenta el conjunto de Julia para $b=1.0$ y se varía $a$ de $-1$ a $3$.

![](JS_b_p1.gif)

Se puede observar que para que existan puntos fijos reales se debe cumplir $a>0$

- A continuación se presenta el conjunto de Julia para $b=-1.0$ y se varía $a$ de $-2$ a $4$.

![](JS_b_m1.gif)

Se puede observar que para que existan puntos fijos reales se debe cumplir $a>-1$

Hola

# Caso disipativo

- En este caso $b\neq|1|$. A continuación se presenta el conjunto de Julia para $b=0.3$ y se varía $a$ de $-0.4$ a $3$.

![](JS_b_03.gif)